In [1]:
import nltk
import json
import random
import pickle
import numpy as np
import tkinter
from tkinter import *
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from keras.models import load_model
model = load_model('model.h5')

res = json.loads(open('intents.json').read())
text = pickle.load(open('words.pkl','rb'))
dataclass = pickle.load(open('classes.pkl','rb'))


def cleaning(data):

    #Tokenize the data
    words =nltk.word_tokenize(data)
    
    #lemmatize the data
    words = [lemmatizer.lemmatize(x.lower()) for x in words]
    
    return words





In [2]:
def bagofwords(data, text, details=True):
    
    
    words = cleaning(data)
    
    # The bag of words for the data
    bow = [0] * len(text)  
    
    for sent in words:
    
        for x,y in enumerate(text):
            
            if y == sent: 
               
                # give 1 to the current word
                bow[x] = 1
                
                if details:
                    
                    print ("found in bag: %s" % w)
                    
    return(np.array(bow))

In [3]:
def prediction(data, model):
    
    # filter out the predictions
    pred = bagofwords(data, text,details=False)
    
    mod = model.predict(np.array([pred]))[0]
    
    error = 0.25
    
    value = [[x,y] for x,y in enumerate(mod) if y > error]
    
    # probability sorting
    value.sort(key=lambda a: a[1], reverse=True)
    
    list = []
    
    for y in value:
        
        
        list.append({"intent": dataclass[y[0]], "probability": str(y[1])})
        
    return list

In [4]:
def response(intent, json):
    
    tags = intent[0]['intent']
    
    intent_list = json['intents']
    
    for x in intent_list:
        
        if(x['tag']== tags):
            
            results = random.choice(x['responses'])
            
            break
            
    return results

In [5]:
def chatbot(message):
    
    intent = prediction(message, model)
    
    mod = response(intent, res)
    
    return mod

In [6]:
def send():
    message = message_box.get("1.0",'end-1c').strip()
    message_box.delete("0.0",END)

    if message != '':
        chat_history.config(state=NORMAL)
        chat_history.insert(END, "you: " + message + '\n\n')
        chat_history.config(fg="Black", font="Tisa")
    
        result = chatbot(message)
        chat_history.insert(END, "Chitti: " + result + '\n\n')
            
        chat_history.config(state=DISABLED)
        chat_history.yview(END)

In [7]:

window = Tk()
window.title("CHITTI")
window.geometry("700x700")
window.resizable(width=FALSE, height=FALSE)

#Chat history 
chat_history = Text(window, bd=0, bg="white", height="7", width="45", font="Tisa",)

chat_history.config(state=DISABLED)

#Scroll bar for the chat history
scroll = Scrollbar(window, command=chat_history.yview)

chat_history['yscrollcommand'] = scroll.set

#Message Box
message_box = Text(window, bd=3, bg="white",width="25", height="7", font="Tisa")


#Send button 
send_button = Button(window, font="Tisa", text="Send", width="10", height="7",bd=3, bg="#4267B2",fg='white',command= send )




chat_history.place(x=6,y=6, height=630, width=670)

message_box.place(x=6, y=600, height=90, width=545)

send_button.place(x=555, y=599, height=90 , width = 140)

scroll.place(x=676,y=6, height=590)

window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\jeshu\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-6-9d3a3dc1014a>", line 10, in send
    result = chatbot(message)
  File "<ipython-input-5-116af713ae16>", line 5, in chatbot
    mod = response(intent, res)
  File "<ipython-input-4-ccbd7abd721f>", line 15, in response
    return results
UnboundLocalError: local variable 'results' referenced before assignment
